### Dense Networks Experiments

This first notebook is to explore the effectiveness of a fully connected neural network to classify tags. The testing will be as follows:

1. Test data which has been subsampled 


In [1]:
import tensorflow as tf
from pydst.rate_limiters import RateLimited 
from pydst.dataproviders import DataProvider
from IPython.display import display, HTML
import datetime
import os
import numpy as np

In [2]:
def show_graph(graph_def, frame_size=(900, 600)):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:{height}px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(height=frame_size[1], data=repr(str(graph_def)), id='graph'+timestamp)
    iframe = """
        <iframe seamless style="width:{width}px;height:{height}px;border:0" srcdoc="{src}"></iframe>
    """.format(width=frame_size[0], height=frame_size[1] + 20, src=code.replace('"', '&quot;'))
    display(HTML(iframe))

In [5]:
def dense(name, inputs, units, fn):
    with tf.variable_scope(name):
        output = tf.layers.dense(inputs=inputs, units=units, name=name, activation=fn, 
                                 kernel_initializer=None) #tf.truncated_normal_initializer())
        return output

def mlp_graph(which_set, rng=None):
    
    graph = tf.Graph()
    outputs = {}
    errors = {}
    accs = {}
    
    with graph.as_default():

        with tf.device('/cpu:0'):
            """trainData = DataProvider(graph=graph, which_set='train', batch_size=10, target_size=50, 
                                     num_samples=50, max_samples=2911, data_depth=40, 
                                     root='../magnatagatune/dataset/fbank/', shape='flat')"""
            trainData = DataProvider(graph=graph, which_set='train', batch_size=10, target_size=50, 
                                     num_samples=10000, shape='flat')
            
            data_batch, targets_batch = trainData.get_data()

        name = 'layer-1'
        outputs[name] = dense(name, data_batch, 2000, tf.nn.relu)                                            

        name = 'layer-2'
        outputs[name] = dense(name, outputs['layer-1'], 1000, tf.nn.relu)

        name = 'layer-3'
        outputs[name] = dense(name, outputs['layer-2'], 500, tf.nn.relu)

        name = 'layer-4'
        outputs[name] = dense(name, outputs['layer-3'], 200, tf.nn.relu)

        name = 'layer-5'
        outputs[name] = dense(name, outputs['layer-4'], 50, tf.nn.relu)

        last_output = outputs['layer-5']
        name = 'error-l5'
        with tf.variable_scope(name):
            errors[name] = tf.losses.sigmoid_cross_entropy(multi_class_labels=targets_batch, logits=last_output)
        tf.summary.scalar(name, errors[name])
        train_step = tf.train.AdamOptimizer().minimize(errors[name])
        
        name = 'accuracy-l5'
        with tf.variable_scope(name):
            accs[name] = tf.reduce_mean(tf.cast(
                tf.equal(tf.argmax(last_output, 1), tf.argmax(targets_batch, 1)), tf.float32))
        tf.summary.scalar(name, accs[name])


        summary_op = tf.summary.merge_all()
        saver = tf.train.Saver()
        
        sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=8))
        
    return sess, graph, summary_op, train_step, trainData, saver, data_batch, targets_batch

In [7]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()


with tf.variable_scope("model_t1") as scope:
    [train_sess, train_graph, train_summary, train_step, trainData, trainSaver, 
                                                                tinputs, ttargets] = mlp_graph('train')
    
#show_graph(train_graph)

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = tf.summary.FileWriter(os.path.join('logs', 'dense_exp', timestamp, 'train'), graph=train_graph)

with train_graph.as_default():

    train_sess.run(tf.global_variables_initializer())
    trainData.enable(train_sess)

    max_iters = trainData.max_num_batches()

    print('Starting iterations')
    for epoch in range(30):

        run_error = 0.
        run_acc = 0.
        print('Epoch: {}'.format(epoch))
        for idx in range(max_iters):

            [_, summary_results] = train_sess.run([train_step, train_summary])          
            writer.add_summary(summary_results, epoch * max_iters + idx)

    trainData.disable(train_sess)

Starting iterations
Epoch: 0


InternalError: Blas GEMM launch failed : a.shape=(10, 10000), b.shape=(10000, 2000), m=10, n=2000, k=10000
	 [[Node: layer-1/layer-1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](batch/_3, layer-1/layer-1/kernel/read)]]
	 [[Node: error-l5/sigmoid_cross_entropy_loss/value/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_556_error-l5/sigmoid_cross_entropy_loss/value", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'layer-1/layer-1/MatMul', defined at:
  File "C:\Users\Mark\Anaconda3\envs\dst\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Mark\Anaconda3\envs\dst\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Mark\Anaconda3\envs\dst\lib\site-packages\ipykernel_launche